# 面向对象高级编程 - 廖雪峰的官方网站

https://www.liaoxuefeng.com/wiki/1016959663602400/1017502538658208

# 使用 `__slots__`

In [8]:
class Student(object):
    pass

In [9]:
s = Student()
s.name = 'Michael' # 动态给实例绑定一个属性
print(s.name)

Michael


In [10]:
def set_age(self, age): # 定义一个函数作为实例方法
    self.age = age

给实例绑定一个方法

In [11]:
from types import MethodType
s.set_age = MethodType(set_age, s) # 给实例绑定一个方法
s.set_age(25) # 调用实例方法
s.age # 测试结果

25

对另一个实例是不起作用的：

In [14]:
s2 = Student() # 创建新的实例
s2.set_age(25) # 调用实例方法

AttributeError: 'Student' object has no attribute 'set_age'

为了给所有实例都绑定方法，可以给 `class` 绑定方法：

通常情况下，下面的`set_score`方法可以直接定义在`class`中，但动态绑定允许我们在程序运行的过程中动态给`class`加上功能，这在静态语言中很难实现。

In [15]:
def set_score(self, score):
    self.score = score

Student.set_score = set_score

In [16]:
s.set_score(100)
s.score

s2.set_score(99)
s2.score

100

99

## 使用__slots__

如果我们想要限制实例的属性怎么办？比如，只允许对 `Student` 实例添加 `name` 和 `age` 属性。

为了达到限制的目的， `Python` 允许在定义 `class` 的时候，定义一个特殊的 `__slots__` 变量，来限制该 `class` 实例能添加的属性：

In [1]:
class Student(object):
    __slots__ = ('name', 'age') # 用tuple定义允许绑定的属性名称

In [2]:
s = Student() # 创建新的实例
s.name = 'Michael' # 绑定属性'name'
s.age = 25 # 绑定属性'age'
s.score = 99 # 绑定属性'score'

AttributeError: 'Student' object has no attribute 'score'

使用 `__slots__` 要注意， `__slots__` 定义的属性仅对当前类实例起作用，对继承的子类是不起作用的：`

In [4]:
class GraduateStudent(Student):
    pass

g = GraduateStudent()
g.score = 9999
g.score

9999

除非在子类中也定义 `__slots__`，这样，子类实例允许定义的属性就是自身的 `__slots__` 加上父类的 `__slots__` 

# 使用@property

在绑定属性时，如果我们直接把属性暴露出去，虽然写起来很简单，但是，没办法检查参数，导致可以把成绩随便改：

In [5]:
s = Student()
s.score = 9999

AttributeError: 'Student' object has no attribute 'score'

为了限制score的范围，可以通过一个 `set_score()` 方法来设置成绩，再通过一个`get_score()` 来获取成绩，这样，在 `set_score()` 方法里，就可以检查参数：

In [6]:
class Student(object):

    def get_score(self):
         return self._score

    def set_score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

In [7]:
s = Student()
s.set_score(60) # ok!
s.get_score()

s.set_score(9999)

60

ValueError: score must between 0 ~ 100!

还记得装饰器 `（decorator）` 可以给函数动态加上功能吗？

对于类的方法，装饰器一样起作用。 `Python` 内置的 `@property` 装饰器就是负责把一个方法变成属性调用的：

`@property` 的实现比较复杂，我们先考察如何使用。

把一个 `getter` 方法变成属性，只需要加上 `@property` 就可以了，此时， `@property` 本身又创建了另一个装饰器 `@score.setter` ，负责把一个 `setter` 方法变成属性赋值，于是，我们就拥有一个可控的属性操作：

In [8]:
class Student(object):

    @property
    def score(self):
        return self._score

    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 ~ 100!')
        self._score = value

In [10]:
s = Student()
s.score = 60 # OK，实际转化为s.set_score(60)
s.score # OK，实际转化为s.get_score()

s.score = 9999

60

ValueError: score must between 0 ~ 100!

只定义 `getter` 方法，不定义 `setter` 方法就是一个 **只读属性** ：

`birth` 是可读写属性，而 `age` 就是一个只读属性，因为 `age` 可以根据 `birth` 和当前时间计算出来

In [12]:
class Student(object):

    @property
    def birth(self):
        return self._birth

    @birth.setter
    def birth(self, value):
        self._birth = value

    @property
    def age(self):
        return 2015 - self._birth

## 练习

请利用 `@property` 给一个 `Screen` 对象加上 `width` 和 `height` 属性，以及一个只读属性 `resolution` 

In [15]:
class Screen():
    
    @property
    def width(self):
        return self._width
    
    @property
    def height(self):
        return self._height
    
    @property
    def resolution(self):
        return self._width * self._height
    

    @width.setter
    def width(self, value):
        self._width = value

    @height.setter
    def height(self, value):
        self._height = value
    



In [16]:
# 测试:
s = Screen()
s.width = 1024
s.height = 768
print('resolution =', s.resolution)
if s.resolution == 786432:
    print('测试通过!')
else:
    print('测试失败!')

resolution = 786432
测试通过!
